Webscraping attempt. Here is the definition of the URL and the use of the requests library to get the webpage

In [1]:
import requests

URL = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats#all_stats_standard'
page = requests.get(URL)


To do this, we are going to use BeautifulSoup. First to import bs4 and make a soup object.

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

After a bit of trekking through the page I have found that "td" objects are those which contain data (although no idea
what the "td" actually stands for). Although you can use "results = results.contents[6].contents" to find the content,
it is much easier to use bs4's "find_all method".

In [3]:
all_data = soup.find_all("td",)
all_teams = soup.find_all(attrs={"data-stat": "squad"})

Let's get a list of all the team names together.

In [4]:
teams = []
for href in all_teams:
    if href.contents[0] != 'Squad':
        teams.append(href.contents[0].get_text())

The find_all method unfortunately for this page gives just one long list. I am fairly sure there will be some what to
load this data straight to a df. For now however, let's get all the unique stats from the long single list.

In [5]:
headers = []

for link in all_data:
    if link.attrs['data-stat'] not in headers:
        headers.append(link.attrs['data-stat'])
    #print(link.attrs['data-stat'], link.contents[0])

Now the trick I have used here is to use pythons defaultdict to make a dictionary of lists and add all values into it.
This is handy for loading straight into a df.

In [6]:
from collections import defaultdict
data_dict = defaultdict(list)

for entry in all_data:
    data_name = entry.attrs['data-stat']
    data_value = entry.contents[0]
    data_dict[data_name].append(str(data_value))


Then this slots quite nicely into a df and then off to a csv

In [7]:

import pandas as pd
data_frame = pd.DataFrame(data_dict)
for i, team in enumerate(teams):
    data_frame = data_frame.rename(index={i: team})
data_frame.to_csv("./team_data.csv")

#for team_stats in all_data:
#    find_stat = re.findall("<td.*?data-stat=\".*?\".*?>.*?td>", team_stats)
    #for i, stat in enumerate(find_stat):
     #   find_stat[i] = re.sub("<td.*=", "", string=stat)
    # print(find_stat)
